## Application allowing to make investments based on stock return predictions

In [23]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import certifi
import json
import requests
import heapq
import yfinance as yf
pd.options.mode.chained_assignment = None  # default='warn'


In [4]:
# List of 20 company symbols for demonstration
watchlist = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'IBM', 'INTC', 'NFLX', 'NVDA',
                   'PYPL', 'CSCO', 'GS', 'JPM', 'BA', 'DIS', 'V', 'WMT', 'C', 'PEP']

# Generate a dictionary with random predicted returns for each company to test the functions
predicted_returns = {symbol: round(random.uniform(-0.05, 0.05), 4) for symbol in watchlist}


In [5]:
def get_current_price(company_name):
    stock = yf.Ticker(company_name)
    return stock.info['currentPrice']

# Now we can use this function to get the current prices for the list of companies
current_prices = {symbol: get_current_price(symbol) for symbol in watchlist}


In [6]:
print(current_prices)

{'AAPL': 185.92, 'MSFT': 388.47, 'GOOGL': 142.65, 'AMZN': 154.62, 'META': 374.49, 'TSLA': 218.89, 'IBM': 165.8, 'INTC': 47.12, 'NFLX': 492.16, 'NVDA': 547.1, 'PYPL': 61.0, 'CSCO': 50.34, 'GS': 377.75, 'JPM': 169.05, 'BA': 217.7, 'DIS': 90.35, 'V': 264.17, 'WMT': 161.32, 'C': 52.62, 'PEP': 167.27}


In [7]:
print(predicted_returns)

{'AAPL': 0.0202, 'MSFT': -0.0488, 'GOOGL': 0.0451, 'AMZN': 0.0371, 'META': -0.0283, 'TSLA': 0.041, 'IBM': 0.0406, 'INTC': -0.0306, 'NFLX': -0.001, 'NVDA': -0.0097, 'PYPL': 0.0367, 'CSCO': 0.0426, 'GS': -0.041, 'JPM': 0.0229, 'BA': -0.0455, 'DIS': 0.0136, 'V': -0.0394, 'WMT': -0.0112, 'C': 0.0204, 'PEP': 0.018}


In [8]:
def select_portfolio(cash, predicted_returns):
    # Sort the returns and take the highest 5
    top_5 = heapq.nlargest(5, predicted_returns.items(), key=lambda item: item[1])
    
    # Calculate the total return of the top 5 stocks
    total_return = sum(return_ for symbol, return_ in top_5)
    
    # Calculate the share of each stock
    shares = {symbol: (return_ / total_return) * cash for symbol, return_ in top_5}
    
    current_prices = {symbol: get_current_price(symbol) for symbol, return_ in top_5}
    
    stock_counts = {symbol: int(cash / current_prices[symbol]) for symbol, cash in shares.items()}
    
    # Create a dataframe
    df = pd.DataFrame(columns=['Company', 'Number of Stocks', 'Initial Price', 'Total Amount'])
    
    for symbol in stock_counts:
        df.loc[len(df)] = {
            'Company': symbol,
            'Number of Stocks': stock_counts[symbol],
            'Initial Price': current_prices[symbol],
            'Total Amount': stock_counts[symbol] * current_prices[symbol]
        }
    remained_cash = cash - df['Total Amount'].sum()

    return df, remained_cash


cash = 100000  # initial investment

portfolio, remained_cash = select_portfolio(cash, predicted_returns)

print(portfolio)
print(f"Cash left: ${remained_cash.round(2)}")


  Company  Number of Stocks  Initial Price  Total Amount
0   GOOGL               153         142.65      21825.45
1    CSCO               410          50.34      20639.40
2    TSLA                90         218.89      19700.10
3     IBM               118         165.80      19564.40
4    AMZN               116         154.62      17935.92
Cash left: $334.73


In [12]:
# FUnction to see the current statement of the portfolio

def current_statement(portfolio):
    old_total = portfolio["Total Amount"].sum()
    # Get the current prices for the stocks in the portfolio
    current_prices = {symbol: get_current_price(symbol) for symbol in portfolio['Company']}
    
    portfolio['Current Price'] = portfolio['Company'].map(current_prices)
    
    # Calculate the new total amount for each stock
    total_amounts = {symbol: current_prices[symbol] * portfolio.loc[portfolio['Company'] == symbol,
                                  'Number of Stocks'].values[0] for symbol in portfolio['Company']}
    
    portfolio['Total Amount'] = portfolio['Company'].map(total_amounts)
    portfolio = portfolio[['Company', 'Number of Stocks', 'Initial Price', 'Current Price', 'Total Amount']]

    total_change_amount = portfolio['Total Amount'].sum() - old_total
    total_change_percent = ((portfolio['Total Amount'].sum() - old_total)/old_total * 100).round(2)
    
    print(f"The change of portfolio volume is ${total_change_amount} ({total_change_percent}%)")
          
    return portfolio

portfolio_current_statement = current_statement(portfolio)
print(portfolio_current_statement)

The change of portfolio volume is $0.0 (0.0%)
  Company  Number of Stocks  Initial Price  Current Price  Total Amount
0   GOOGL               153         142.65         142.65      21825.45
1    CSCO               410          50.34          50.34      20639.40
2    TSLA                90         218.89         218.89      19700.10
3     IBM               118         165.80         165.80      19564.40
4    AMZN               116         154.62         154.62      17935.92


In [10]:
new_predictions = {symbol: round(random.uniform(-0.05, 0.05), 4) for symbol in watchlist}
print(new_predictions)

{'AAPL': 0.0336, 'MSFT': -0.0409, 'GOOGL': -0.0392, 'AMZN': -0.0047, 'META': -0.025, 'TSLA': -0.0343, 'IBM': 0.0404, 'INTC': 0.0174, 'NFLX': 0.0135, 'NVDA': 0.0365, 'PYPL': -0.0389, 'CSCO': 0.0394, 'GS': 0.0199, 'JPM': -0.0345, 'BA': -0.0027, 'DIS': -0.0281, 'V': 0.0082, 'WMT': 0.0204, 'C': -0.0238, 'PEP': -0.0446}


### Definition of the portfolio update function below

Now that we have got a function to create a new portfolio, and to see the current statement, I want to have a function to update our portfolio based on the new predictions of the return.
Here are the steps we need to do:
1. We need to sort the new_predictions list to find out which companies suggest the highest return for the next period, and get the top 5 companies
2. Calculate the hypothetical cash, that we are going to invest. For that we need to be sure 30% of our portfolio remains unchanged, which means we are going to take the current sum of amount of our portfolio, calculate 70% of it, and consider it as a hypothetical amount of cash that we have to invest in the new top 5 companies
3. Now we are going to select which stocks we preserve in our portfolio. For this purpose we are going to find out the new predicted returns for our current portfolio companies, sort them to see the ones with highest returns, and take 2 highest ones.
4. Having the 30% of our current total amount of money, we are going to decide how many stocks we are going to leave in our current portfolio out of those 2 companies, based on their predicted return. For example if we have $30000 that need to be remained unchanged, and the predicted returns for AAPL and META are 0.12 and 0.24, then for the AAPL we can calculate the number of stocks left in the portfolio as 300000*0.12/(0.12+0.24) / initial price of AAPL, to see how many complete stocks we are going to leave in out portfolio anyway.
5. Now that we have got 30% of out portfolio volume unchanged, as we fixed the number of stocks to leave in our portfolio for 2 companies, we are going to decide how many stocks we are going to buy with 70% amount of our hypothetical cash, in the sape principle as we created the portfolio, that is taking the new prediction as weights by which we are going to distribute out 70% cash in the 5 companies.
6. Now knowing how many stocks we need for the rest 70% of our portfolio, we are going to perform sell and buy opreations. First we are going to sell those stocks that we don't need for ouu new portfolio, to get the cash to buy the new stocks. In this purpose we are going to take the number of old stocks (the 30%) we are going to leave in our portfolio anyway, and create a new df with those 2 companies and their number of stocks. After taking those shares out of our current portfolio, we are going to subtract the same amount of stocks out of the new portfolio as much as possible. For example if we are going to leave 120 AAPl and 30 META stocks unchanged in our new portfolio, and if we have calculated that we need to have 130 AAPL, 10 META, 60 TESLA, 7 GOOGLE and 3 C stocks, we need to take out the previous 2 stocks, leaving that we need to have stocks_to_have 10AAPL, 0 META, 60 TESLA, 7 GOOGLE and 3C new stocks.
7. Knowing what we are going to buy, we are going to compare this list with the old portfolio out of which the 30% of the 2 companies were taken out. We could just sell all the stocks left, but as the buying and selling process costs money, we are going to see which stocks we are going to leave unchanged in the old portfolio and which we are going to buy. So if we now see that in the old portfolio there are 20 AAPL, 40 META, 70 TESLA, 30 NIKE and 10 GOOGLE stocks left after taking out the 120 AAPl and 30 META stocks, now we will compare that with the stocks_to_have list, we will First decide what stocks to sell.
In this particular case we can see that we need 10 AAPL but have 20, so we need to sell 10 AAPL, then we see we need 0 META but we have 40 META, so we need to sell 40 META, then we see we need 60 TESLA, but we have 70 TESLA, so we need to sell 10 TESLA, then we see we need 0 NIKE but we have 30 NIKE, so we are going to sell them all, we need 7 GOOGLE but we have 10 google so we need to sell 3 google.
Note that we sell the unnecessery stocks so that we had cash to buy what we need to add to our portfolio, for example 3 C stocks.

In [26]:
def update_portfolio_based_on_predictions(portfolio, new_predictions, remained_cash):
    # Step 1: Get the top 5 companies
    top_5 = heapq.nlargest(5, new_predictions.items(), key=lambda item: item[1])
    
    df = current_statement(portfolio)

    # Step 2: Calculate the hypothetical cash
    total_amount = df['Total Amount'].sum()
    hypothetical_cash_70 = (total_amount) * 0.7 + remained_cash
    hypothetical_cash_30 = (total_amount) * 0.3

    # Step 3: Select which 2 stocks to preserve in our portfolio
    current_portfolio_predictions = {symbol: new_predictions[symbol] for symbol in df['Company']}
    preserved_stocks = heapq.nlargest(2, current_portfolio_predictions.items(), key=lambda item: item[1])

    # Step 4: Decide how many stocks to leave in our current portfolio unchanged
    # There is a bug in here in case when the predicted return is negative
    preserved_shares = {}
    for symbol, return_ in preserved_stocks:
        calculated_shares = int((return_ / sum(return_ for symbol, return_ in preserved_stocks)) * hypothetical_cash_30 / df.loc[df['Company'] == symbol, 'Current Price'].values[0])
        current_shares = df.loc[df['Company'] == symbol, 'Number of Stocks'].values[0]
        preserved_shares[symbol] = min(calculated_shares, current_shares)
    
    df_preserved = df[df['Company'].isin([symbol for symbol in preserved_shares])].copy()
    df_preserved['Number of Stocks'] = df_preserved['Company'].map(preserved_shares)
    df_preserved['Total Amount'] = df_preserved['Number of Stocks'] * df_preserved['Current Price']

    # Step 5: Calculate the number of shares that we are eager to sell
    # Merge the two dataframes on 'Company'
    merged_df = pd.merge(df, df_preserved, on='Company', how='outer', suffixes=('', '_preserved'))

    # Fill NaN values with 0
    merged_df.fillna(0, inplace=True)

    # Calculate the number of shares that we are eager to change
    merged_df['Number of Stocks'] = merged_df['Number of Stocks'] - merged_df['Number of Stocks_preserved']
    merged_df['Total Amount'] = merged_df['Number of Stocks'] * merged_df['Current Price']

    # Drop the preserved columns
    merged_df.drop(columns=['Number of Stocks_preserved', 'Initial Price_preserved', 'Current Price_preserved', 'Total Amount_preserved'], inplace=True)

    # Keep only the rows where 'Number of Stocks' is not 0
    ready_to_sell_df = merged_df[merged_df['Number of Stocks'] != 0].copy()
    
    # Step 6: Decide how many stocks we need to have with 70% of our cash
    shares_to_have_70 = {symbol: int((return_ / sum(return_ for symbol, return_ in top_5)) * hypothetical_cash_70 / get_current_price(symbol)) for symbol, return_ in top_5}
    new_shares_to_have_70 = shares_to_have_70.copy()
    # Step 7: In case we already have a part of the amount of need-to-buy stocks in the ready_to_sell list,
    # we subtract them from the need-to-buy list

    for symbol in shares_to_have_70:
        if symbol in ready_to_sell_df['Company'].values:
            shares_to_have_70[symbol] = max(0, shares_to_have_70[symbol] - ready_to_sell_df.loc[ready_to_sell_df['Company'] == symbol, 'Number of Stocks'].values[0])
    stocks_to_buy = shares_to_have_70.copy()

    
    # Step 8: Define which stocks to sell
    stocks_to_sell = {}
    for symbol in ready_to_sell_df['Company'].values:
        if symbol in new_shares_to_have_70:
            stocks_to_sell[symbol] = max(0, ready_to_sell_df.loc[ready_to_sell_df['Company'] == symbol, 'Number of Stocks'].values[0] - new_shares_to_have_70[symbol])
        else:
            stocks_to_sell[symbol] = ready_to_sell_df.loc[ready_to_sell_df['Company'] == symbol, 'Number of Stocks'].values[0]
    
    old_portfolio = df.copy()
    
    # Step 9: Selling the stocks_to_sell stocks of the old portfolio df with the current prices, adding new_cash to remained_cash
    new_cash = remained_cash
    for symbol, num_stocks in stocks_to_sell.items():
        current_price = get_current_price(symbol)  # replace with your function to get current price
        new_cash += num_stocks * current_price

        # Remove the sold stocks from the portfolio
        df.loc[df['Company'] == symbol, 'Number of Stocks'] -= num_stocks
        df.loc[df['Company'] == symbol, 'Total Amount'] = df.loc[df['Company'] == symbol, 'Number of Stocks'] * current_price
    
    print(f"initial cash {remained_cash}")
    print(f"cash after selling the stocks: {new_cash}")
    
#    # Remove the rows where 'Number of Stocks' is 0 (optional)
#    df = df[df['Number of Stocks'] != 0]
    
    # Step 10: Buying the stocks for the new portfolio using new_cash and stocks_to_buy list, and adding them to the df
    for symbol, num_stocks in stocks_to_buy.items():
        current_price = get_current_price(symbol)
        new_cash -= num_stocks * current_price

        # Add the bought stocks to the portfolio
        if symbol in df['Company'].values:
            df.loc[df['Company'] == symbol, 'Number of Stocks'] += num_stocks
            df.loc[df['Company'] == symbol, 'Total Amount'] = df.loc[df['Company'] == symbol, 'Number of Stocks'] * current_price
        else:
            df.loc[len(df)] = {
                'Company': symbol,
                'Number of Stocks': num_stocks,
                'Initial Price': current_price,
                'Current Price': current_price,
                'Total Amount': num_stocks * current_price
            }


    print(f"cash after buying the new stocks: {new_cash}")


    
# Continue the code from here:

    return old_portfolio, df_preserved, ready_to_sell_df, new_shares_to_have_70, stocks_to_buy, stocks_to_sell, df, new_cash
    
    
old_portfolio, df_preserved, ready_to_sell_df, new_shares_to_have_70, stocks_to_buy, stocks_to_sell, new_portfolio, new_cash = update_portfolio_based_on_predictions(portfolio, new_predictions, remained_cash)
print("Old portfolio:")
print(old_portfolio)
print("Stocks to preserve:")
print(df_preserved)
print("Stocks that we are ready to sell:")
print(ready_to_sell_df)
print("New stocks that we need to have in 70%:")
print(new_shares_to_have_70)
print("New stocks that we need to buy:")
print(stocks_to_buy)
print("Old stocks that we need to sell:")
print(stocks_to_sell)
print("The new portfolio after update:")
print(new_portfolio)

The change of portfolio volume is $0.0 (0.0%)
initial cash 334.7299999999959
cash after selling the stocks: 59796.2
cash after buying the new stocks: 454.6799999999912
Old portfolio:
  Company  Number of Stocks  Initial Price  Current Price  Total Amount
0   GOOGL               153         142.65         142.65      21825.45
1    CSCO               410          50.34          50.34      20639.40
2    TSLA                90         218.89         218.89      19700.10
3     IBM               118         165.80         165.80      19564.40
4    AMZN               116         154.62         154.62      17935.92
Stocks to preserve:
  Company  Number of Stocks  Initial Price  Current Price  Total Amount
1    CSCO               293          50.34          50.34      14749.62
3     IBM                91         165.80         165.80      15087.80
Stocks that we are ready to sell:
  Company  Number of Stocks  Initial Price  Current Price  Total Amount
0   GOOGL             153.0         142.65 

In [28]:
new_portfolio["Total Amount"].sum() + new_cash

100000.0